[과제] https://news.naver.com/ 사이트에서 4개의 뉴스 카테고리를 선택해서 뉴스기사를 크롤링 한 후 다음을 수행하세요.

- 뉴스기사 카테고리별 저장(파일 or DB)
- 텍스트 전처리 및 피처 벡터화
- 모델링 및 평가(성능개선 포함)

In [26]:
import os
import sys
import urllib.request
client_id = "Xm1WkS4jX4BHrHrIYVwb"
client_secret = "Mo1eRhZvLT"

encText = urllib.parse.quote("정치")
url = "https://openapi.naver.com/v1/search/news?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
"lastBuildDate": "Fri, 18 Mar 2022 12:27:23 +0900",
"total": 7136243,
"start": 1,
"display": 10,
"items": [
{
"title": "뉴욕주, 직장내 괴롭힘·차별 없앤다",
"originallink": "https://news.koreadaily.com/2022/03/17/society/generalsociety/20220317202305042.html",
"link": "https://news.koreadaily.com/2022/03/17/society/generalsociety/20220317202305042.html",
"description": "결국 사임한 쿠오모 전 주지사는 최근엔 본인이 ‘<b>정치</b>적 공격의 희생자’라며 혐의를 부인하고, <b>정치</b>적으로 재기하려는 움직임을 보이고 있다. 한편, 호컬 주지사는 이날 성희롱 문제를 해결할 수 있는 무료 핫라인을... ",
"pubDate": "Fri, 18 Mar 2022 12:24:00 +0900"

},
{
"title": "군위군, 3~4월에 처리하겠다던, 군위군 대구 편입 ‘안갯속’",
"originallink": "https://www.viva100.com/main/view.php?key=20220318010004648",
"link": "https://www.viva100.com/main/view.php?key=20220318010004648",
"description": "지역 <b>정치</b>권의 적극적인 역할이 필요하다. ‘대구·경북 통합 신공항 건설을 조속 추진하겠다’는 ‘윤석열... 갈등 조정을 위해 지역 <b>정치</b>권이 최대한 빨리 새 정부에 협조를 구해야 한다는 목소리가 높아지고 있다. 군위=",
"pubDate": "Fri, 18 Mar 2022 12:24:00 +0900"

},
{
"title": "&quot;정보 집중 검경 권한 남용 가능성&quot;…법무부·경찰 인사검

In [38]:
import json
data = json.loads(response_body.decode('utf-8'))

{'lastBuildDate': 'Fri, 18 Mar 2022 12:27:23 +0900',
 'total': 7136243,
 'start': 1,
 'display': 10,
 'items': [{'title': '뉴욕주, 직장내 괴롭힘·차별 없앤다',
   'originallink': 'https://news.koreadaily.com/2022/03/17/society/generalsociety/20220317202305042.html',
   'link': 'https://news.koreadaily.com/2022/03/17/society/generalsociety/20220317202305042.html',
   'description': '결국 사임한 쿠오모 전 주지사는 최근엔 본인이 ‘<b>정치</b>적 공격의 희생자’라며 혐의를 부인하고, <b>정치</b>적으로 재기하려는 움직임을 보이고 있다. 한편, 호컬 주지사는 이날 성희롱 문제를 해결할 수 있는 무료 핫라인을... ',
   'pubDate': 'Fri, 18 Mar 2022 12:24:00 +0900'},
  {'title': '군위군, 3~4월에 처리하겠다던, 군위군 대구 편입 ‘안갯속’',
   'originallink': 'https://www.viva100.com/main/view.php?key=20220318010004648',
   'link': 'https://www.viva100.com/main/view.php?key=20220318010004648',
   'description': '지역 <b>정치</b>권의 적극적인 역할이 필요하다. ‘대구·경북 통합 신공항 건설을 조속 추진하겠다’는 ‘윤석열... 갈등 조정을 위해 지역 <b>정치</b>권이 최대한 빨리 새 정부에 협조를 구해야 한다는 목소리가 높아지고 있다. 군위=',
   'pubDate': 'Fri, 18 Mar 2022 12:24:00 +0900'},
  {'title': '&quot;정보 집중 검경 권한 

In [44]:
# 뉴스별 링크
data['items'][1]['link']

'https://www.viva100.com/main/view.php?key=20220318010004648'

In [58]:
import os
import sys
import urllib.request
import pandas as pd
import json
import re

client_id = "Xm1WkS4jX4BHrHrIYVwb"
client_secret = "Mo1eRhZvLT"

querys = [urllib.parse.quote('정치'),urllib.parse.quote('IT/과학'),urllib.parse.quote('생활/문화'),urllib.parse.quote('경제')]

idx = 0
display = 100
start = 1
end = 500

web_df = pd.DataFrame(columns = ['Title','Link','Description','Label'])
for query in querys:
    for start_index in range(start,end,display):
        url = "https://openapi.naver.com/v1/search/news?query=" + query \
        + '&display=' + str(display) \
        + '&start=' + str(start_index)
        
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']
            for item_index in range(0, len(items)):
                remove_tag = re.compile('<.*?>')
                title = re.sub(remove_tag,'', items[item_index]['title'])
                link = items[item_index]['link']
                description = re.sub(remove_tag,'',items[item_index]['description'])
                label = query
                web_df.loc[idx] = [title,link,description,label]
                idx += 1
            
        else:
            print("Error Code:" + rescode)

In [59]:
web_df

,Title,Link,Description,Label
0,"임병하 약사, 경북도의원 출사표 &quot;준비됐다&quot;",https://www.kpanews.co.kr/article/show.asp?idx...,임병하 약사의 정치 도전은 이번이 두번째다. 2007년 급작스레 진행된 보궐선거에 ...,%EC%A0%95%EC%B9%98
1,"38노스 &quot;북, 러시아산 원유 수입 등 수혜 가능성&quot;",https://news.naver.com/main/read.naver?mode=LS...,러시아의 우크라이나 침공에 따른 신냉전 격화 속에 북한이 러시아산 원유를 값싸게 수...,%EC%A0%95%EC%B9%98
2,"윤호중 비대위원장, 당 내 사퇴 압박에 ‘사실상 거부’⋯민주당 내홍 커지나...",https://www.asiatime.co.kr/article/20220318500080,"그는 “우리 비대위가 당 혁신을 힘 있게 추진하고 나아가 정치개혁, 대장동 특검, ...",%EC%A0%95%EC%B9%98
3,"&quot;17분 푸틴 연설, 우크라 침공 계획대로 되지 않는단 신호&quot;",https://news.naver.com/main/read.naver?mode=LS...,타티아나 스타노바야 정치 분석 전문가는 자신의 소셜미디어(SNS)를 통해 &quot...,%EC%A0%95%EC%B9%98
4,"양산시, 2022년도'여성인재학당'수강생 모집",http://www.enewstoday.co.kr/news/articleView.h...,"교육기간은 오는 5월부터 11월까지 5개월 과정으로, 여성의 정치·경제·사회참여, ...",%EC%A0%95%EC%B9%98
...,...,...,...,...
1995,"남부발전, 국제정세 대응 위한 탄소중립 정책포럼 개최",https://www.straightnews.co.kr/news/articleVie...,남부발전은 최근 부산 본사 대강당에서 조성봉 숭실대학교 경제학과 교수를 초청해 경영...,%EA%B2%BD%EC%A0%9C
1996,최정우 회장 &quot;100년 기업 원년…친환경 미래소재 기업으로 거듭&quot;,https://news.mtn.co.kr/news-detail/20220318105...,제공하는 경제·환경·사회적 가치를 포괄한다&quot;며 이 같이 말했다.최 회장은 ...,%EA%B2%BD%EC%A0%9C
1997,"전국 최초 특별지자체, ‘부산·울산·경남 특별연합’ 출범한다",https://news.naver.com/main/read.naver?mode=LS...,"경남도 관계자는 “산업, 인재, 공간의 3대 분야에 초점을 둬 각 분야를 육성하고 ...",%EA%B2%BD%EC%A0%9C
1998,"[중심고을충주 단신] 충북 북부권 혁신지원센터, 충주에 둥지 外",http://www.psnews.co.kr/news/articleView.html?...,"센터에는 충청북도 혁신기관(충북과학기술혁신원, 충북테크노파크, 충북연구원, 충북창조...",%EA%B2%BD%EC%A0%9C


AttributeError: 'str' object has no attribute 'decode'

In [54]:
web_df.loc[2].Link

'http://www.newswatch.kr/news/articleView.html?idxno=58520'